Парсер TripAdvisor. Часть 2, собирает данные со страницы объекта.


Последнее обновление: 10 апреля 2022

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/' + input()
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

#s=Service('C:\Users\boris\anaconda3\chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

### Собираю отзывы

In [ ]:
# ссылка на страницу TripAdvisor

url = 'https://www.tripadvisor.ru/Attraction_Review-g298484-d301574-Reviews-Lenin_s_Mausoleum-Moscow_Central_Russia.html'

In [ ]:
driver.get(url)
sleep(5)

# выбираю Все языки
driver.find_element(By.CLASS_NAME, 'fIZlc').find_elements(By.TAG_NAME, 'button')[1].click()
driver.find_elements(By.TAG_NAME, 'li[role="none"]')[24].click()

In [ ]:
data = [] # список для данных

for i in tqdm(range(138)):
    
    review_blocks = driver.find_elements(By.TAG_NAME, 'div[data-automation="reviewCard"]')
    
    for review in review_blocks:
        user_name = review.find_element(By.CLASS_NAME, 'cjhIj').find_element(By.TAG_NAME, 'a').text
        user_profile = (review.find_element(By.CLASS_NAME, 'cjhIj').find_element(By.TAG_NAME, 'a')
                        .get_attribute('href'))

        city_publ = review.find_element(By.CLASS_NAME, 'ddOtn').text

        try:
            user_review_number = review.find_element(By.CLASS_NAME, 'fhriQ').text

        except:
            user_review_number = ''

        city = review.find_element(By.CLASS_NAME, 'ddOtn').text.replace(user_review_number, '')
        user_score = (review.find_element(By.TAG_NAME, 'svg[class="RWYkj d H0"]')
                      .get_attribute('aria-label'))

        visit_date = review.find_element(By.CLASS_NAME, 'eRduX').text

        review_all = []
        for part in review.find_elements(By.CLASS_NAME, 'NejBf'):
            review_all += [part.text]

        review_title = review_all[0]
        review_text = review_all[1]



        try:
            review.find_element(By.TAG_NAME, 'button').find_element(By.LINK_TEXT, 'Подробнее').click()

        except:
            pass


        data.append([user_name, city, user_score, visit_date, review_title, review_text,
                     user_profile, user_review_number])
        
        
    driver.find_elements(By.CLASS_NAME, 'eRhUG')[-1].click()
    sleep(6)
        
    

In [ ]:
head = ['user_name', 'city', 'user_score', 'visit_date', 'review_title', 'review_text',
        'user_profile', 'user_review_number']

work_data = pd.DataFrame(data, columns=head)

In [ ]:
work_data.shape

In [ ]:
work_data.sample(3)

In [ ]:
work_data.drop_duplicates().shape